In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/tsp.csv')
df.head()

,City1,City2,Minutes
0,Barcelona,Vic,85
1,Vic,Barcelona,80
2,Barcelona,Girona,100
3,Girona,Barcelona,95
4,Barcelona,Tarragona,90


In [3]:
city_ids = {i: city for i, city in enumerate(df['City1'].unique())}
city_ids

{0: 'Barcelona',
 1: 'Vic',
 2: 'Girona',
 3: 'Tarragona',
 4: 'Lleida',
 5: 'Figueres',
 6: 'Reus',
 7: 'Manresa',
 8: 'Sabadell'}

In [4]:
def create_individual(num_cities):
    """Create a random individual (route)."""
    route = list(range(num_cities))
    random.shuffle(route)
    return route

In [38]:
def get_city_name(city_id):
    """Get the city name from the city id."""
    return city_ids[city_id]

def compute_fitness(individual):
    """Calculate the total distance of the route."""
    total_distance = 0
    for i in range(len(individual)):
        current_city_name = get_city_name(individual[i])
        next_city_name = get_city_name(individual[(i + 1) % len(individual)])

        # Find the distance between the two cities
        total_distance += df[(df['City1'] == current_city_name) & (df['City2'] == next_city_name)]['Minutes'].iloc[0]
    
    return total_distance

In [9]:
def select_parents(population, fitnesses):
    """Select two parents based on their fitness. Less fitness -> higher chance to be selected."""
    max_fitness = max(fitnesses)
    inverted_fitnesses = [max_fitness - fitness for fitness in fitnesses]
    total_inverted_fitness = sum(inverted_fitnesses)
    selection_probs = [fitness / total_inverted_fitness for fitness in inverted_fitnesses]
    return random.choices(population, k=2, weights=selection_probs)

In [10]:
def crossover(parent1, parent2):
    """Perform order crossover."""
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [None] * len(parent1)
    child[start:end] = parent1[start:end]
    parent2_filtered = [item for item in parent2 if item not in child]
    child[:start] = parent2_filtered[:start]
    child[end:] = parent2_filtered[start:]
    return child

In [33]:
def mutate(individual, mutation_rate):
    """Perform swap mutation."""
    for i in range(len(individual)):
        if np.random.random() < mutation_rate:
            swap_with = np.random.randint(0, len(individual))
            individual[i], individual[swap_with] = individual[swap_with], individual[i]
    return individual

In [36]:
def run_genetic_algorithm(seq_length, population_size, generations, mutation_rate, keep_best):
    """
    Run the genetic algorithm.

    seq_length: length of the bit string
    population_size: number of individuals in the population
    generations: number of generations to run
    mutation_rate: probability of mutation for each gene
    keep_best: whether to keep the best individual from the previous generation

    Return the final population.
    """

    # Initialize population
    population = [create_individual(seq_length) for _ in range(population_size)]
    fitnesses = [compute_fitness(individual) for individual in population]

    for generation in range(generations):
        # Create new generation through selection, crossover, and mutation

        # Initialize new population
        new_population = []
        if keep_best:
            sorted_fitness_idxs = np.argsort(fitnesses, axis=0)
            new_population.append(population[sorted_fitness_idxs[0]])
            new_population.append(population[sorted_fitness_idxs[1]])

        for _ in range((population_size - len(new_population)) // 2):
            # Select two parents
            parent1, parent2 = select_parents(population, fitnesses)

            # Crossover parents
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent1, parent2)

            # Mutate children
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            # Add children to new population
            new_population.extend([child1, child2])

        # Update population and compute new fitnesses
        population = new_population
        fitnesses = [compute_fitness(individual) for individual in population]

        # Print out the best fitness in this generation
        best_fitness = min(fitnesses)
        print(f"Generation {generation}, Best Fitness: {best_fitness}")

    return population

In [43]:
# Run the GA
final_population = run_genetic_algorithm(seq_length=len(city_ids), population_size=100, generations=50, mutation_rate=0.2, keep_best=True)

Generation 0, Best Fitness: 620
Generation 1, Best Fitness: 620
Generation 2, Best Fitness: 605
Generation 3, Best Fitness: 605
Generation 4, Best Fitness: 605
Generation 5, Best Fitness: 605
Generation 6, Best Fitness: 605
Generation 7, Best Fitness: 605
Generation 8, Best Fitness: 605
Generation 9, Best Fitness: 605
Generation 10, Best Fitness: 605
Generation 11, Best Fitness: 605
Generation 12, Best Fitness: 605
Generation 13, Best Fitness: 550
Generation 14, Best Fitness: 550
Generation 15, Best Fitness: 550
Generation 16, Best Fitness: 550
Generation 17, Best Fitness: 550
Generation 18, Best Fitness: 550
Generation 19, Best Fitness: 550
Generation 20, Best Fitness: 550
Generation 21, Best Fitness: 550
Generation 22, Best Fitness: 550
Generation 23, Best Fitness: 550
Generation 24, Best Fitness: 550
Generation 25, Best Fitness: 550
Generation 26, Best Fitness: 550
Generation 27, Best Fitness: 550
Generation 28, Best Fitness: 550
Generation 29, Best Fitness: 550
Generation 30, Best 

In [44]:
fitnesses = [compute_fitness(individual) for individual in final_population]
best_idx = np.argmin(fitnesses)
best_individual = final_population[best_idx]

print(f"Best individual: {best_individual}")
print(f"Fitness: {fitnesses[best_idx]}min")
print()

print("Itinerary:")
for i in range(len(best_individual)):
    current_city = get_city_name(best_individual[i])
    next_city = get_city_name(best_individual[(i + 1) % len(best_individual)])
    distance = df[(df['City1'] == current_city) & (df['City2'] == next_city)]['Minutes'].iloc[0]
    print(f'{current_city} --> {next_city}: {distance}min')

Best individual: [8, 2, 5, 1, 7, 4, 3, 6, 0]
Fitness: 550min

Itinerary:
Sabadell --> Girona: 75min
Girona --> Figueres: 50min
Figueres --> Vic: 95min
Vic --> Manresa: 45min
Manresa --> Lleida: 85min
Lleida --> Tarragona: 55min
Tarragona --> Reus: 20min
Reus --> Barcelona: 95min
Barcelona --> Sabadell: 30min
